In [ ]:
from scipy.optimize import linprog
import seaborn as sns

from scipy.stats import kurtosis, skew
import pandas as pd
from rcv_distribution import *
from rcv_dimensionality import *
from voting_rules import *
import os

In [ ]:
def calculate_intervals(numbers):
    midpoints = []
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            midpoints.append((numbers[i] + numbers[j])/2)

    intervals = []
    midpoints = sorted(midpoints)
    for i in range(len(midpoints)):
        for j in range(i + 1, len(midpoints)):
            intervals.append((midpoints[i], midpoints[j]))
    return midpoints, intervals

In [ ]:
def ballot_to_num(ballot, normalized_distances):
    b_num = []
    for c in ballot:
        b_num.append(normalized_distances[c])
    return b_num

In [ ]:
def solve_lp(b_num, midpoints, intervals, n):
    
    obj = [1]
    lhs_ineq = []
    rhs_ineq = []
    e = 0.000000001
    for i in range(len(b_num)):
        for j in range(i + 1, len(b_num)):
            mid = (b_num[i]+b_num[j])/2
            if (b_num[i] > b_num[j]):
                lhs_ineq.append([-1])
                rhs_ineq.append(-(e+mid))
            if (b_num[i] < b_num[j]):
                lhs_ineq.append([1])
                rhs_ineq.append(e+mid)
    bnd = [0, n-1]
    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd,
              method="revised simplex")
    return (opt["success"] is True)
        

    

In [ ]:
def get_interval_consistent_ballots(ballots, normalized_distances):
    """
    Args:
        ballots: a dictionary of ballots paired with the number of voters voting fo that ranking.
        normalized_distances: 
    Return:
        consistent_ballots: a dictionary of all consistent ballots.
        gamma: the fraction of the consistent ballots.
        
    """
    midpoints, intervals = calculate_intervals(list(normalized_distances.values()))

    total = 0
    consistent = 0
    consistent_ballots = {}
    for b in ballots:
        if (len(b) != 0):
            total += ballots[b]
            if (len(b) <= 2):
                consistent += ballots[b]
                consistent_ballots[b] = ballots[b]
            else:
                b_num = ballot_to_num(b, normalized_distances)
                lp = solve_lp(b_num, midpoints, intervals, len(normalized_distances))
                if (lp):
                    consistent_ballots[b] = ballots[b]
                    consistent += ballots[b]
    
    gamma = consistent/total
    return consistent_ballots, gamma
        

In [ ]:
def get_bullet_votes(ballots):
    total = 0
    bullet = 0
    for b in ballots:
        if (len(b) > 0):
            total += ballots[b]
        if (len(b) == 1):
            bullet += ballots[b]
    return bullet/total

In [ ]:
winners = pd.DataFrame(columns=["filename", "diff", "interval gamma", "#candidates","IRV1", "Condorcet1", "Plurality1", "Approval1" "IRV2", "Condorcet2", "Plurality2", "Approval2"])
directory = "dataverse_files"
count = 0
for filename in os.listdir(directory):
    count += 1
    if (count % 50):
        print(count, " ", filename)
    try:
        csv =  os.path.join(directory, filename)
        ballots, candidates = parse_election_data(csv)

        # Perform the RCV analysis
        test = perform_rcv_analysis(csv, n_runs=1000)
        mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names = test

        # Print the normalized distances between candidates and plot the MDS analysis
        normalized_distances = get_distances_normalized(most_common_order, mds_1d_coordinates, candidate_names)
        
        #running voting rules in the original election 
        org_election = voting_rules(ballots, candidates)
        irv1 = org_election.irv()
        condorcet1 = org_election.condorcet()
        plurality1 = org_election.plurality()
        approval1 = org_election.approval()

        #getting consistent ballots using the interval approach
        consistent_ballots, gamma = get_interval_consistent_ballots(ballots, normalized_distances)
        
        #running voting rules on only consistent ballots
        election = voting_rules(consistent_ballots, candidates)
        irv2 = election.irv()
        condorcet2 = election.condorcet()
        plurality2 = election.plurality()
        approval2 = election.approval()

        diff = []
        if (irv1 != irv2):
            diff.append("IRV")
        if (condorcet1 != condorcet2):
            diff.append("Condorcet")
        if (plurality1 != plurality2):
            diff.append("Plurality")
        if (approval1 != approval2):
            diff.append("Approval")
        winners = winners.append({'filename': filename, 'diff': diff, "interval gamma": gamma, "#candidates": len(candidates), 'IRV1': irv1, 'Condorcet1': condorcet1,
                                'Plurality1': plurality1, 'Approval1': approval1,
                                'IRV2': irv2, "Condorcet2": condorcet2, 
                                'Plurality2':plurality2, 'Approval2': approval2}, ignore_index=True)


    except:
        print(filename, " threw an exception")
winners.to_csv('interval_winners.csv', index=False)
    